In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corpus2/pre_proc_train.csv
/kaggle/input/corpus2/pre_proc_test_time.csv
/kaggle/input/corpus2/pre_proc_test_company.csv


In [2]:
os.listdir('/kaggle/input')

['corpus2']

In [3]:
!pip install torch torchvision transformers pillow

In [4]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
fail_img = 0

In [6]:
from PIL import Image
import requests
from io import BytesIO
import torch.nn.functional as F
from torch.utils.data import Dataset

ref_img = Image.new('RGB', (224, 224), color='black')

class CLIPDataset(Dataset):
    def __init__(self, csv_file, processor):
        self.data = pd.read_csv(csv_file)
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_url = self.data.iloc[idx]['full_image_url']  # Extract full image URL
        text = self.data.iloc[idx]['formatted_text']
        target = self.data.iloc[idx]['likes']  # Extract the number of likes

        # Load image from URL
        global fail_img,ref_img
        image = load_image(img_url)
        if list(image.getdata()) == list(ref_img.getdata()) :
            # Handle the case when the image couldn't be loaded
            print(f"Skipping row {idx} due to invalid image.")
            fail_img+=1
#             return create_placeholder_image()

        # Process image and text
        encoding = self.processor(text=[text], images=image, return_tensors="pt", padding=True, truncation=True)
#         print(f"Text tokens shape: {encoding['input_ids'].shape}")
#         print(f"Image shape: {encoding['pixel_values'].shape}")

        return encoding['input_ids'].squeeze(0), encoding['pixel_values'].squeeze(0), target
    


def compute_loss(logits, targets):
    # Assuming 'targets' contains the number of likes
    return F.mse_loss(logits, targets)



def create_placeholder_image():
    return Image.new('RGB', (224, 224), color='black')

def load_image(image_url):
#     response = requests.get(image_url)
#     image = Image.open(BytesIO(response.content)).convert("RGB")
#     return image
    try:
        # Make a request to get the image content
        response = requests.get(image_url)
        # Check if the request was successful
        if response.status_code == 200:
            # Try to open the image
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image = image.resize((224, 224))
            return image
        else:
            print(f"Failed to retrieve image from {image_url}. HTTP Status code: {response.status_code}")
            return create_placeholder_image()
    except Exception as e:
        # Print the error and return None if there's an issue loading the image
        print(f"Error loading image from {image_url}: {e}")
        return None



In [7]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    
    batch = [item for item in batch if item is not None]
    if not batch:
        return None
    try:
        input_ids = [item[0] for item in batch]
        pixel_values = [item[1] for item in batch]
        targets = [item[2] for item in batch]
#         print(f"Targets before conversion: {targets}")
#         print(f"Targets types: {[type(t) for t in targets]}")
        targets = torch.as_tensor([item[2] for item in batch])
    
    except Exception as e:
        print("Error loading Batch")
        print(f"Input IDs shape: {[id.shape for id in input_ids]}")
        print(f"Pixel Values shape: {[pixel.shape for pixel in pixel_values]}")
#         print(f"Targets shape: {targets.shape}")
        
        print()
#         return None
        
    # Pad the input_ids (text tokens) to ensure equal length in the batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=processor.tokenizer.pad_token_id)

    # Stack the pixel values (images), ensure they are all resized properly beforehand
    pixel_values = torch.stack(pixel_values)  # Assuming all images are resized to same dimensions (e.g., 224x224)

    return input_ids, pixel_values, targets

In [8]:
from torch.utils.data import DataLoader

# Initialize dataset and dataloader
dataset = CLIPDataset(csv_file='/kaggle/input/corpus2/pre_proc_train.csv', processor=processor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True,collate_fn=custom_collate_fn)

In [9]:
# for batch in dataloader:
#     print(batch)
#     break  # Just to check the first batch
# for batch in dataloader:
#     for item in batch:
#         print("Item content:", item)  # Print each item in the batch
#     break  # Check the first batch
# for batch in dataloader:
#     input_ids, pixel_values, targets = batch
#     print(f"Input IDs shape: {[id.shape for id in input_ids]}")
#     print(f"Pixel Values shape: {[pixel.shape for pixel in pixel_values]}")
#     print(f"Targets shape: {targets.shape}")
#     break

In [10]:
def average_logits(logits):
    """
    Calculate the average of each row in a logits tensor.
    
    Parameters:
    - logits (torch.Tensor): A tensor of shape [16, 16].
    
    Returns:
    - torch.Tensor: A tensor of shape [16] containing the average of each row.
    """
    # Ensure the input is a 2D tensor
    if logits.dim() != 2 or logits.size(0) != 16 or logits.size(1) != 16:
        raise ValueError("Input tensor must be of shape [16, 16].")
    
    # Calculate the average across the second dimension (columns)
    averages = torch.mean(logits, dim=1)  # This will give shape [16]
    
    return averages

In [11]:
# logits_per_image = torch.randn(16, 16)  # Example tensor
# averaged_logits = average_logits(logits_per_image)
# print(averaged_logits.shape)  # Should print: torch.Size([16])
# print(averaged_logits)  # Print the averaged values

In [12]:
import wandb
wandb.login(key="f720772ccda55cec1d742ae1e1e279cb3fa597bf")

wandb.init(project='Adobe-NLP-PS',name='CLIP_run_2')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: gary_hornbill (gary_hornbill-indian-institute-of-technology-ism-dhanbad). Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.18.3


wandb: Run data is saved locally in /kaggle/working/wandb/run-20241009_190359-paagdh9n
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run CLIP_run_2


wandb: ⭐️ View project at https://wandb.ai/gary_hornbill-indian-institute-of-technology-ism-dhanbad/Adobe-NLP-PS


wandb: 🚀 View run at https://wandb.ai/gary_hornbill-indian-institute-of-technology-ism-dhanbad/Adobe-NLP-PS/runs/paagdh9n


In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 50

for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids, pixel_values,target = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        
        target = target.float() 
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, pixel_values=pixel_values)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

        # Calculate your custom loss function
        # Here you can implement a regression loss based on the number of likes
        # e.g., use Mean Squared Error for regression
        avg_logits_per_image = average_logits(logits_per_image) #Converts [16,16] tensor to [16]

        loss = compute_loss(avg_logits_per_image, target)  # Define how to compute your loss

        # Backpropagation
        try:
            loss.backward()
        except Exception as e:
#             print(logits_per_image)
            print(avg_logits_per_image.dtype)  # Should be Float
            print(target.dtype) 
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    print(fail_img)
    wandb.log({'epoch': epoch},
              {'train_loss': loss.item()}
             )


Failed to retrieve image from https://pbs.twimg.com/media/Ejai0-LXsAAag5A?format=jpg&name=large. HTTP Status code: 404
Skipping row 238635 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/media/DSqZktyWsAEXQTq?format=jpg&name=large. HTTP Status code: 404
Skipping row 164244 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/media/Dkwo1JZXoAAvrN1?format=jpg&name=large. HTTP Status code: 404
Skipping row 185164 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/media/Epn7e6-XYAoYUYp?format=jpg&name=large. HTTP Status code: 404
Skipping row 199735 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/media/EWzfvbkWkAMbZ3U?format=jpg&name=large. HTTP Status code: 404
Skipping row 85210 due to invalid image.


...

Failed to retrieve image from https://pbs.twimg.com/media/DnQIiE2X0AEUtji?format=jpg&name=large. HTTP Status code: 404
Skipping row 168210 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/tweet_video_thumb/D1i2_YxX4AAePiF.jpg. HTTP Status code: 404
Skipping row 234836 due to invalid image.


Failed to retrieve image from https://pbs.twimg.com/media/EQ0350vWAAA6K8O?format=jpg&name=large. HTTP Status code: 404
Skipping row 224252 due to invalid image.


Skipping row 176753 due to invalid image.


Skipping row 266757 due to invalid image.
